In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import random   

np.random.seed(42)
random.seed(42)

desire = 'Eat'
test_data_path = f'data/test/{desire}.csv'
test_df = pd.read_csv(test_data_path, index_col=False)
test_df.head(2)

,person_id,travel_purpose,target_amenity,mode,distance_miles,duration_minutes,age,individual_income,household_size,family_structure,vehicles,cypher
0,5.570035e+18,Eat,F&B Eatery/Drinking Places,Walking,0.0,0.0,29,53672,3_person,family_single,1,\n MATCH (p:Person)\n WHERE (p.a...
1,7.600486e+18,Eat,F&B Eatery/Snack and Nonalcoholic Beverage Bars,Walking,0.0,0.0,26,0,3_person,nonfamily_single,0,\n MATCH (p:Person)\n WHERE (p.a...


In [ ]:
from model import ChoiceModel
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd

num_sample = 900
num_threads = 3

data = []

choice_model = ChoiceModel(
    data_dir='data', desire=desire, sample_num=num_sample, skip_init=True)

def process_row(row):
    try:
        profile = row[['person_id', 'age', 'individual_income',
                        'household_size', 'family_structure', 'vehicles', 'cypher']]
        profile = profile.to_dict()
        response = choice_model.infer(
            profile=profile, mode='experiment',in_parallel=True)
        print("get result:", response)
        return response
    except Exception as e:
        print(e)
        pass

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    for idx,row in tqdm(test_df.iterrows(), total=len(test_df)):
        future = executor.submit(process_row, row)
        response = future.result()
        print("get response:", response)
        if response:
            data.append(response)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
data_df = pd.DataFrame(data, columns=['person_id', 'profile', 'top_k', 'desire', 'city', 'cypher',
                                      'amenity_recommendation', 'amenity_llm_choice', 'amenity_final_choice',
                                      'mode_recommendation', 'mode_llm_choice', 'mode_final_choice'])